In [1]:
# Setup
api_key = open('alpha_vantage_apikey.txt').read()

# 1. Company's time series data
Get time series data from representative companies in different industry, using API, store in separate json files

In [2]:
import json
import requests
def make_company_json(company_stock_name):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={company_stock_name}&apikey={api_key}'

    # Send the API request and retrieve the response
    response = requests.get(url)
    data = response.json()
    fileName = company_stock_name + '.json'
    with open(fileName, 'w') as f:
        json.dump(data["Time Series (Daily)"], f)

### Information Technology - AAPL (APPLE), MSFT(MICROSOFT)

In [3]:
make_company_json('AAPL')
make_company_json('MSFT')

### Energy -  Exxon Mobil (XOM), Chevron (CVX)

In [4]:
make_company_json('XOM')
make_company_json('CVX')

### Health Care - Johnson & Johnson (JNJ), Eli Lilly & Co. (LLY) 

In [6]:
make_company_json('JNJ')
make_company_json('LLY')

Convert json to db

In [7]:
import os
import sqlite3
def read_data(filename):
    current_folder = globals()['_dh'][0]
    full_path = os.path.join(current_folder, filename)
    f = open(full_path)
    file_data = f.read()
    f.close()
    json_data = json.loads(file_data)
    return json_data
def open_database(db_name):
    current_folder = globals()['_dh'][0]
    full_path = os.path.join(current_folder, db_name)
    conn = sqlite3.connect(full_path)
    cur = conn.cursor()
    return cur, conn

company_list = ['AAPL','MSFT','CVX','XOM', 'JNJ','LLY']
json_data = {}
for company in company_list:
    json_data[company] = read_data(company + '.json')
cur, conn = open_database('company_stock.db')

Load all json file into database, under company.db
Insert 25 rows each time, {id, date, closing stock price}

In [8]:
for company in company_list:
    query = f"DROP TABLE IF EXISTS {company}"
    cur.execute(query)
conn.commit()

In [13]:
def make_stock_close_tb(data, cur, conn,company):
    # store date and close price into a list
    dateL,closeL = [],[]
    for date in data:
        dateL.append(date)
        closeL.append(data[date]["4. close"])
    # Check if the Company table exist
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{company}'"
    cur.execute(query)
    table_exists = cur.fetchone() is not None

    if table_exists:
        # if the table exists 
        # dates are being inserted in reverse order (most recent date = 1st row)
        query = f"SELECT MIN(date) FROM {company}"
        cur.execute(query)
        most_not_recent_date = cur.fetchone()[0]
        startIndex = dateL.index(most_not_recent_date) + 1
    else:
        startIndex = 0

    #Create the table and insert approapriate data
    createTB = f"CREATE TABLE IF NOT EXISTS {company} (id INTEGER PRIMARY KEY, date DATE UNIQUE, close DOUBLE)"
    cur.execute(createTB)
    #Insert 25 rows at a time
    for i in range(startIndex,min(startIndex+25,100)):
        query = f"INSERT OR IGNORE INTO {company} (id, date, close) VALUES (?,?,?)"
        cur.execute(query, (i + 1, dateL[i], closeL[i]))
    conn.commit()


for company in company_list:
    make_stock_close_tb(json_data[company],cur,conn,company)

# 2.S&P Index
Using Yahoo API and downloaded a year s&p index csv file, convert to data base

In [14]:
import yfinance as yf
import pandas as pd

sp500 = yf.Ticker('^GSPC')
sp500_hist = sp500.history(period='6mo')
sp500_hist = sp500_hist.sort_values(by='Date', ascending=False)
sp500_hist.to_csv('sp500_6m.csv',index_label=None)

In [15]:
cur.execute("DROP TABLE IF EXISTS SP500")

In [19]:
import csv

# Check if the table exists
cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='SP500'")
table_exists = cur.fetchone() is not None
idx = 0
if table_exists:
    # If the table exists, find the row number of the most recent date
    cur.execute("SELECT MIN(date) FROM SP500")
    most_not_recent_date2 = cur.fetchone()[0]
    for date in sp500_hist.index:
        if (str(date)[:10] == most_not_recent_date2[:10]):
            idx = sp500_hist.index.get_loc(date) + 1
            break
cur.execute("CREATE TABLE IF NOT EXISTS SP500 (date DATE UNIQUE, close DOUBLE)")

# Read the CSV file and insert the remaining rows into the database
with open('sp500_6m.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row
    i = 0
    for row in reader:
        if i >= idx:
            cur.execute("INSERT OR IGNORE INTO SP500 (date, close) VALUES (?,?)",(row[0][:10],row[4]))
        if i == idx + 24:
            break
        i += 1

# Commit changes and close the database connection
conn.commit()